In [20]:
%load_ext autoreload
%autoreload 2
import torch
import jax

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Torch Model

In [2]:
# %cd ~/InternVL/classification/
# from models.build import build_model
# # initialize config
# import ml_collections
# config_dict = {
#     "MODEL": {
#         "TYPE": "intern_vit_6b",
#         "NUM_CLASSES": 0,
#         "DROP_PATH_RATE": 0.0,
#         "INTERN_VIT_6B": {
#             "PATCH_SIZE": 16,
#             "EMBED_DIM": 768,
#             "NUM_HEADS": 12,
#             "MLP_RATIO": 4,
#             "QKV_BIAS": True,
#             "INIT_VALUES": 1e-6,
#             "QK_NORMALIZATION": False,
#             "DEPTH": 12,
#             "USE_FLASH_ATTN": False,
#             "PRETRAIN_SIZE": 224,
#             "PRETRAINED": None,
#             "CLS_TARGET": "clip_projector",
#             "HEAD_NORM_TYPE": "none",
#             "FREEZE_VIT": False,
#         },
#     },
#     "DATA": {
#         "IMG_SIZE": 224
#     },
#     "TRAIN": {
#         "USE_CHECKPOINT": False
#     }
# }
# config = ml_collections.ConfigDict(config_dict)
# model = build_model(config=config)
%cd ~/unilm/beit/
from modeling_pretrain import beit_base_patch16_224_8k_vocab
model = beit_base_patch16_224_8k_vocab(pretrained=False)
# Input Initialization
dummy_img = torch.randn(1, 3, 224, 224)
bool_masked_pos = torch.randn(1,196).bool()
out = model.forward_features(dummy_img,bool_masked_pos)
out.shape, model

/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/austinwang/unilm/beit


/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/austinwang/unilm/beit/modeling_finetune.py:379: UserWarning: Overwriting beit_base_patch16_224 in registry with modeling_finetune.beit_base_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def beit_base_patch16_224(pretrained=False, **kwargs):
/home/austinwang/unilm/beit/modeling_finetune.py:388: UserWarning: Overwriting beit_base_patch16_384 in registry with modeling_finetune.beit_base_patch16_384. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def beit_base_patch16_384(pretrained=False, **kwargs):
/home/austinwang/unilm/beit/modeling_finetune.py:397: UserWarning: Overwriti

(torch.Size([1, 197, 768]),
 VisionTransformerForMaskedImageModeling(
   (patch_embed): PatchEmbed(
     (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
   )
   (pos_drop): Dropout(p=0.0, inplace=False)
   (blocks): ModuleList(
     (0-11): 12 x Block(
       (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
       (attn): Attention(
         (qkv): Linear(in_features=768, out_features=2304, bias=False)
         (attn_drop): Dropout(p=0.0, inplace=False)
         (proj): Linear(in_features=768, out_features=768, bias=True)
         (proj_drop): Dropout(p=0.0, inplace=False)
       )
       (drop_path): Identity()
       (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
       (mlp): Mlp(
         (fc1): Linear(in_features=768, out_features=3072, bias=True)
         (act): GELU(approximate='none')
         (fc2): Linear(in_features=3072, out_features=768, bias=True)
         (drop): Dropout(p=0.0, inplace=False)
       )
     )
   )
   (norm): Lay

# Trucated normalization comparison

In [23]:
# Flax
import flax
import jax.numpy as jnp
class flax_foo(flax.linen.Module):
    feature_dim: int
    kernel_init: flax.linen.initializers.Initializer = flax.linen.initializers.truncated_normal(0.02, dtype=jnp.float32, lower=-0.02, upper=0.02)
    bias_init: flax.linen.initializers.Initializer = flax.linen.initializers.zeros
    def setup(self):
        self.dense = flax.linen.Dense(features=self.feature_dim, kernel_init=self.kernel_init, bias_init=self.bias_init)
    def __call__(self, x):
        return self.dense(x)

# torch 
from timm.models.layers import trunc_normal_
def trunc_normal(param): return trunc_normal_(param, mean=0.0, std=0.02, a=-0.02, b=0.02)
class torch_foo(torch.nn.Module):
    def __init__(self, feature_dim: int):
        super().__init__()
        self.dense = torch.nn.Linear(feature_dim, feature_dim)
        trunc_normal(self.dense.weight)
        torch.nn.init.zeros_(self.dense.bias)
    def forward(self, x):
        return self.dense(x)


In [36]:
# Input Initialization
feature_dim = 3
x = torch.randn(1, feature_dim, requires_grad=False)
x_flax = jnp.array(x)

# model initialization
flax_model = flax_foo(feature_dim)
params = flax_model.init(jax.random.PRNGKey(0), x_flax)
torch_model = torch_foo(feature_dim)
torch_params = torch_model.state_dict()


In [37]:
params['params']['dense']['kernel'], torch_params['dense.weight']

(Array([[-4.0258612e-05,  2.2024987e-04, -1.4944360e-04],
        [-3.4164346e-04,  3.4291650e-05, -3.5881568e-04],
        [-8.0378231e-05, -2.9115484e-04, -1.3574921e-04]], dtype=float32),
 tensor([[-0.0051,  0.0075,  0.0189],
         [-0.0121,  0.0095,  0.0100],
         [-0.0084, -0.0161,  0.0034]]))

In [42]:
params['params']['dense']['kernel'].std().item(), torch_params['dense.weight'].std().item()

(0.0001775238779373467, 0.011768573895096779)

In [ ]:

# forward pass
flax_out = flax_model.apply(params, x_flax)
torch_out = torch_model(x)